<a href="https://colab.research.google.com/github/jacmal/Crypto-Anaysis/blob/main/Crypto_Analysis_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ANALIZA RYNKU KRYPTOWALUT

## Opis

# INSTALACJA BIBLIOTEK

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# IMPORT BIBLIOTEK

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# pobrane biblioteki
import yfinance as yf

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.signal import argrelextrema

# USTAWIENIA

In [ ]:
MAIN_PATH = "/content/drive/MyDrive/Projects/4_Crypto_Analysis"
DIR_NAME = "CMA"

In [ ]:
main_dir_path = os.path.join(MAIN_PATH, DIR_NAME)
data_dir_path = os.path.join(main_dir_path, "Market_Data")

## config.py

In [ ]:
import json
import os
import configparser

import pandas as pd


class Config:
    def __init__(self):
        self.existsc = False

    def exists(self): 
        if self.pathc == None:
            self.existsc = False
            print('Path is type None')
        elif type(self.pathc) == str and type(self.namec) == str and len(self.namec) > 0:
            self.path_config = os.path.join(self.pathc, self.namec)
            if os.path.exists(self.path_config):
                self.existsc = True
                print('Config exists!')
                return self.existsc
            else:
                self.existsc = False
                print('Config not exists')
                return self.existsc
        else:
            print('wrong config path')
            return self.existsc

    def _save_json(self):
        print(self.config)
        print(self.path_config)
        with open(self.path_config, 'w') as json_file:
            json.dump(self.config, json_file)

    def _load_json(self):
        with open(self.path_config) as json_file:
            data = json.load(json_file)
        print(data)
        self.config = data

    def _save_ini(self):
        sections = []
        config_file = configparser.ConfigParser()
        
        for i in list(self.config.keys()):
            if self.config[i][0] not in sections:
                sections.append(self.config[i][0])
                config_file.add_section(self.config[i][0])
            
            config_file.set(self.config[i][0], i, self.config[i][1])

        with open(self.path_config, 'w') as ini_file:
            config_file.write(ini_file)
            ini_file.flush()
            ini_file.close()

    def _load_ini(self):
        config_file = configparser.ConfigParser()
        config_file.read(self.path_config)
        
        config_keys = list(config_file.keys())
        #config_keys.remove('DEFAULT')
        self.config = {}

        for s in config_keys:
            for n in config_file[s]:
                print(s, n, config_file[s][n])
                self.config[n] = (s, config_file[s][n])
            print()

################################################################################ 
    
    def set_path(self, config_path, config_name, mode="ini"):
        self.pathc = config_path
        self.namec = config_name
        self.mode = mode
        
        name = self.namec + "." + self.mode
        self.path_config = os.path.join(self.pathc, name)

    def reset(self, section_name="MAIN", add_main=True):
        self.config = {}
        if add_main == True:
            self.config["name"] = (section_name, self.namec)
            self.config["path"] = (section_name, self.pathc)
            self.config["mode"] = (section_name, self.mode)

    def addp(self, name, section, val):
        self.config[name] = (section, val)

    def save(self):
        if self.mode == "ini":
            print('ini mode')
            self._save_ini()
        elif self.mode == "json":
            print('json mode')
            self._save_json()
        print(self.path_config)
        print("Plik konfiguracyjny zapisany!")

    def load_last(self):
        if self.mode == "ini":
            self._load_ini()
        elif self.mode == "json":
            self._load_json()

    def load_path(self, config_path, config_name):
        self.set_path(config_path, config_name, mode='json')
        self.reset()
        
        if self.mode == "ini":
            print('ini mode')
            self._load_ini()
        
        elif self.mode == "json":
            print('json mode')
            self._load_json()
        
        print(self.path_config)
        print("Plik konfiguracyjny wczytany!")


    def getp(self, name):
        return self.config[name][1]

    
    def return_cf_data(self, data_type="df"):
        data_dict = {}
        for k, v in self.config.items():
            print(k, v)

# INSTALACJA

In [ ]:
def install():
    os.mkdir(main_dir_path)
    os.mkdir(data_dir_path)

In [ ]:
if not os.path.exists(main_dir_path):
    install()

else:
    print("Aplication is allready installed!")

# INICJALIZACJA

# POBIERANIE DANYCH RYNKU KRYPTOWALUT

## BINANCE

### binance_api.py

In [ ]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import time

import configparser
import binance

from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from date_time_functions import set_datetime, datetime_now, datetime_clean


class BinanceAPI:
    def __init__(self, test_mode=True):
        self.client = None
        self.last_order = None
        self.test_mode = test_mode
        
        # Binance kline columns list
        self.columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 
                        'Close Time', 'Quote asset volume', 'Number of trades', 
                        'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

    
    def _client_init(self):
        self.client = Client(self.api_key, self.sec_key, tld="com", )

################################################################################

    def load_api_keys_file(self, keys_file_path):
        config_keys = configparser.ConfigParser()
        config_keys.read(keys_file_path)

        self.api_key = config_keys['API_KEYS']['api_key']
        self.sec_key = config_keys['API_KEYS']['sec_key']
        
        self._client_init()
    
    
    def test_connection(self):
        print(self.client.ping())
    
    
    def get_account_info(self):
        return self.client.get_account()

    
    def get_balances(self):
        account = self.client.get_account()
        #balances = account["balances"]
        time_updete = pd.to_datetime(account["updateTime"], unit="ms")
        df_balances = pd.DataFrame(account["balances"])
        
        df_balances["free"] = pd.to_numeric(df_balances.free, errors="coerce")
        df_balances["locked"] = pd.to_numeric(df_balances.locked, errors="coerce")
        
        return (df_balances, time_updete)
    
    def get_balance(self, symbol, free_lock="free"):
        return float(self.client.get_asset_balance(asset=symbol)[free_lock])

    
    def get_status(self):
        return self.client.get_system_status()

    
    def get_fee(self):
        return self.client.get_trade_fee()

    
    def get_account_snapshot(self, snap_type="SPOT"):
        return self.client.get_account_snapshot(type=snap_type)


    def get_market_data(self, symbol, interval, start_date, history="False", datetime_idx=True):
        '''
        if history == True:
            start_date = ''
        '''

        if interval == '1DAY':
            hist_data = self.client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1DAY, start_date)
        elif interval == '1HOUR':
            hist_data = self.client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1HOUR, start_date)
        elif interval == '15MINUTE':
            hist_data = self.client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start_date)
        elif interval == '1MINUTE':
            hist_data = self.client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date)
        
        #print(hist_data)
        
        df = pd.DataFrame(hist_data)
        df.columns = self.columns
             
        df['Open Time'] = pd.to_datetime(df['Open Time']/1000, unit='s')
        df['Close Time'] = pd.to_datetime(df['Close Time']/1000, unit='s')

        numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, axis=1)

        df = df.iloc[:, :6]
        
        if datetime_idx == True:
            df.set_index("Open Time", inplace=True, drop=True)
            return df
        else:
            return df
    
    def market_order(self, osymbol, oside, oquantity):
        self.last_order = self.client.create_order(symbol=osymbol, side=oside, quantity=oquantity, type="MARKET")
        return self.last_order

## TODO TITLE: FREE CRYPTO API